In [1]:
import pandas as pd

train_path='/media/data2/rjsdn/dacon/airplane_customer/'

train = pd.read_csv(train_path + 'train.csv')

train.head()

,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
0,1,Female,disloyal Customer,22,Business travel,Eco,1599,3,0,3,...,4,5,4,4,4,5,4,0,0.0,0
1,2,Female,Loyal Customer,37,Business travel,Business,2810,2,4,4,...,5,5,4,2,1,5,2,18,18.0,0
2,3,Male,Loyal Customer,46,Business travel,Business,2622,1,1,1,...,4,4,4,4,5,4,3,0,0.0,1
3,4,Female,disloyal Customer,24,Business travel,Eco,2348,3,3,3,...,3,2,4,5,3,4,3,10,2.0,0
4,5,Female,Loyal Customer,58,Business travel,Business,105,3,3,3,...,4,4,4,4,4,4,5,0,0.0,1


In [2]:
def check_missing_col(df):
    missing_col=[]
    for c in df.columns:
        missing_values = sum(df[c].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'na col: {c}')
            print(f'num of na: {missing_values}')
            missing_col.append([c,df[c].dtype])
    if len(missing_col)==0:
        print('NO NA')
    return missing_col

missing_col=check_missing_col(train)

NO NA


In [3]:
train_x = train.drop(['id'],axis=1)
train_y = train.target

def make_label_map(df):
    label_maps={}
    for c in df.columns:
        if df[c].dtype=='object':
            label_map={'unknown':0}
            for i,key in enumerate(df[c].unique()):
                label_map[key] = i+1
            label_maps[c]=label_map
    return label_maps

def label_encoder(df,label_map):
    for c in df.columns:
        if df[c].dtype=='object':
            df[c] = df[c].map(label_map[c])
            df[c] = df[c].fillna(0)
    return df

label_maps = make_label_map(train_x)
train_x = label_encoder(train_x,label_maps)

train_x.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
0,1,1,22,1,1,1599,3,0,3,3,...,4,5,4,4,4,5,4,0,0.0,0
1,1,2,37,1,2,2810,2,4,4,4,...,5,5,4,2,1,5,2,18,18.0,0
2,2,2,46,1,2,2622,1,1,1,1,...,4,4,4,4,5,4,3,0,0.0,1
3,1,1,24,1,1,2348,3,3,3,3,...,3,2,4,5,3,4,3,10,2.0,0
4,1,2,58,1,2,105,3,3,3,3,...,4,4,4,4,4,4,5,0,0.0,1


In [4]:
from sklearn.preprocessing import MinMaxScaler

num_features = ['Age','Flight Distance','Departure Delay in Minutes','Arrival Delay in Minutes']

scaler = MinMaxScaler()
train_x[num_features] = scaler.fit_transform(train_x[num_features])
train_x.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
0,1,1,0.205479,1,1,0.226501,3,0,3,3,...,4,5,4,4,4,5,4,0.000000,0.000000,0
1,1,2,0.410959,1,2,0.403807,2,4,4,4,...,5,5,4,2,1,5,2,0.015957,0.016143,0
2,2,2,0.534247,1,2,0.376281,1,1,1,1,...,4,4,4,4,5,4,3,0.000000,0.000000,1
3,1,1,0.232877,1,1,0.336164,3,3,3,3,...,3,2,4,5,3,4,3,0.008865,0.001794,0
4,1,2,0.698630,1,2,0.007760,3,3,3,3,...,4,4,4,4,4,4,5,0.000000,0.000000,1


In [5]:
from tqdm import tqdm


In [6]:
from pycaret.classification import *
setup_clf=setup(data=train_x,target='target',n_jobs=3)

,Description,Value
0,session_id,1067
1,Target,target
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(3000, 23)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,18
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
topn = compare_models(sort='Accuracy',n_select=4)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9252,0.9809,0.9317,0.9358,0.9335,0.8480,0.8486,0.0670
et,Extra Trees Classifier,0.9161,0.9774,0.9300,0.9225,0.9260,0.8293,0.8299,0.3720
gbc,Gradient Boosting Classifier,0.9123,0.9737,0.9207,0.9243,0.9221,0.8218,0.8228,0.2660
rf,Random Forest Classifier,0.9119,0.9750,0.9275,0.9177,0.9221,0.8205,0.8217,0.4230
lr,Logistic Regression,0.8947,0.9637,0.9072,0.9074,0.9067,0.7858,0.7872,0.3080
svm,SVM - Linear Kernel,0.8933,0.0000,0.9215,0.8953,0.9073,0.7815,0.7845,0.0290
ada,Ada Boost Classifier,0.8918,0.9585,0.9022,0.9069,0.9041,0.7800,0.7811,0.1040
ridge,Ridge Classifier,0.8880,0.0000,0.9055,0.8979,0.9013,0.7719,0.7730,0.0510
lda,Linear Discriminant Analysis,0.8866,0.9618,0.9005,0.8998,0.8997,0.7692,0.7703,0.0710
knn,K Neighbors Classifier,0.8776,0.9474,0.8980,0.8869,0.8922,0.7505,0.7510,0.1100


In [8]:
top5

[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_jobs=3, num_leaves=31, objective=None,
                random_state=1067, reg_alpha=0.0, reg_lambda=0.0, silent='warn',
                subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
 ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=None, max_features='auto',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=3,
                      oob_score=False, random_state=1067, verbose=0,
                      warm_start=Fal

In [9]:
tuned_top5 = [tune_model(i) for i in top5]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8810,0.9658,0.9412,0.8615,0.8996,0.7541,0.7586
1,0.9048,0.9692,0.9160,0.9160,0.9160,0.8061,0.8061
2,0.8810,0.9449,0.8487,0.9352,0.8899,0.7610,0.7652
3,0.9095,0.9760,0.9412,0.9032,0.9218,0.8146,0.8155
4,0.9190,0.9608,0.9244,0.9322,0.9283,0.8354,0.8354
5,0.9095,0.9614,0.9328,0.9098,0.9212,0.8151,0.8154
6,0.9095,0.9720,0.9153,0.9231,0.9191,0.8165,0.8165
7,0.8952,0.9504,0.8814,0.9286,0.9043,0.7887,0.7900
8,0.9000,0.9750,0.9068,0.9145,0.9106,0.7971,0.7972
9,0.8947,0.9645,0.9068,0.9068,0.9068,0.7859,0.7859


In [11]:
blend = blend_models(estimator_list=tuned_top5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9000,0.9825,0.9748,0.8657,0.9170,0.7924,0.8012
1,0.9381,0.9748,0.9328,0.9569,0.9447,0.8744,0.8748
2,0.8714,0.9594,0.8571,0.9107,0.8831,0.7406,0.7422
3,0.9333,0.9863,0.9412,0.9412,0.9412,0.8643,0.8643
4,0.9333,0.9793,0.9412,0.9412,0.9412,0.8643,0.8643
5,0.9190,0.9708,0.9328,0.9250,0.9289,0.8350,0.8350
6,0.9238,0.9819,0.9322,0.9322,0.9322,0.8452,0.8452
7,0.9095,0.9706,0.9068,0.9304,0.9185,0.8169,0.8172
8,0.9048,0.9832,0.9237,0.9083,0.9160,0.8061,0.8062
9,0.9187,0.9788,0.9322,0.9244,0.9283,0.8344,0.8344


In [12]:
final_model = finalize_model(blend)

[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 

# TEST

In [13]:
test = pd.read_csv('/media/data2/rjsdn/dacon/airplane_customer/test.csv')
test_missing_col = check_missing_col(test)

NO NA


In [14]:
test_x = test.drop(['id'],axis=1)

test_label_map = make_label_map(test_x)
test_x = label_encoder(test_x,test_label_map)
test_x[num_features] = scaler.transform(test[num_features])


In [15]:
pred = predict_model(final_model,data=test_x)

In [18]:
pred.Label

0       1
1       0
2       1
3       1
4       1
       ..
1995    0
1996    1
1997    0
1998    1
1999    1
Name: Label, Length: 2000, dtype: int64

In [ ]:
import numpy as np

In [19]:
sub = pd.read_csv('/media/data2/rjsdn/dacon/airplane_customer/sample_submission.csv')
sub.target = pred.Label
sub.to_csv('submission.csv',index=False)
    
    


[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will 

/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_intercept_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute average_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute average_intercept_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/s


[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.4, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will 

/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute standard_intercept_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute average_coef_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute average_intercept_ was deprecated in version 0.23 and will be removed in 0.25.
  warnings.warn(msg, category=FutureWarning)
/home/rjsdn/.conda/envs/ML/lib/python3.7/site-packages/s


[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] bagging_fraction is set=0.4, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will 